In [2]:
import pandas as pd

df = pd.read_csv("./data/cleaned_creditcard.csv")
df.head(3)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Class,scaled_amount,scaled_time
0,-1.364601,-0.350611,3.007412,0.511529,0.194795,0.865765,-0.749215,0.655023,0.655106,-0.839261,...,0.823843,-0.343019,-0.460763,0.634386,-0.376703,0.110085,0.085196,0,0.181653,0.384086
1,-2.434004,3.225947,-6.596282,3.593161,-1.079452,-1.739741,-0.047420,0.301424,-1.779434,-5.836453,...,-0.419178,0.157436,-0.714849,0.468859,-0.348522,0.420036,-0.327643,1,4.758611,0.529517
2,0.419326,-0.783233,1.376835,3.110438,-1.335870,-0.011701,-0.038907,0.001432,-0.072302,0.310870,...,-0.122397,-0.202255,0.956296,0.123122,-0.115909,-0.017333,0.103011,0,4.538531,-0.624902


In [3]:
X = df.drop('Class', axis=1)
Y = df['Class']

In [4]:
from sklearn.model_selection import train_test_split

# split the dataset into dependent and independent features
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [5]:
# Turn the values into an array for feeding the classification algorithms.
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

Model Building
1. Logistic Regression
2. KNN
3. SVC
4. DecisionTreeClassifier
5. RandomForestClassifier

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [7]:
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier()
}

In [8]:
from sklearn.model_selection import cross_val_score

for key, classifier in classifiers.items():
    classifier.fit(x_train,y_train)
    training_score = cross_val_score(classifier, x_train, y_train,cv=10)
    print()
    print(classifier, " : Training accuracy score is", round(training_score.mean(),2)*100, "%")


LogisticRegression()  : Training accuracy score is 95.0 %

KNeighborsClassifier()  : Training accuracy score is 93.0 %

SVC()  : Training accuracy score is 94.0 %

DecisionTreeClassifier()  : Training accuracy score is 89.0 %

RandomForestClassifier()  : Training accuracy score is 95.0 %


In [9]:
# Prediction

for key, classifier in classifiers.items():
    prediction_score = classifier.predict(x_test)
    print(classifier, " : Prediction accuracy score is", round(prediction_score.mean(),2)*100, "%")

LogisticRegression()  : Prediction accuracy score is 53.0 %
KNeighborsClassifier()  : Prediction accuracy score is 52.0 %
SVC()  : Prediction accuracy score is 52.0 %
DecisionTreeClassifier()  : Prediction accuracy score is 56.00000000000001 %
RandomForestClassifier()  : Prediction accuracy score is 54.0 %


# Observation

Clearly we can see this is the case of overfitting. Here bias is hight but varience is low. This leads to overfitting.

To overcome overfitting, Lets use grid search cv for getting best parameter

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score


classifiers = {
    'log_reg': LogisticRegression(solver='liblinear'),
    'knears_neighbors': KNeighborsClassifier(),
    'svc': SVC(),
    'tree_clf': DecisionTreeClassifier(),
    'forest_clf': RandomForestClassifier()
}

def bestparameter():
    param_list = {
        'log_reg': {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
        'knears_neighbors': {"n_neighbors": list(range(2, 5, 1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},
        'svc': {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
        'tree_clf': {"criterion": ["gini", "entropy"], "max_depth": list(range(2, 4, 1)),
                     "min_samples_leaf": list(range(5, 7, 1))},
        'forest_clf': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    }

    for clf_name, clf in classifiers.items():
        random_search = RandomizedSearchCV(clf, param_distributions=param_list[clf_name], n_iter=10, cv=5,
                                           scoring='accuracy', random_state=42)
        random_search.fit(x_train, y_train)
        print(f"Best Parameters for {clf_name}: {random_search.best_params_}")

        # Update the best estimator in classifiers dictionary
        classifiers[clf_name] = random_search.best_estimator_



def validation_score():

    print("\n\n==============================================================")
    for clf_name, clf in classifiers.items():
        clf_score = cross_val_score(clf, x_train, y_train, cv=5)
        
        print(f'Cross Validation Score of {clf} : \n-- {round(clf_score.mean() * 100, 2)}%')
    print("\n================================================================")

    # Assuming x_test and y_test are your test data
    for clf_name, clf in classifiers.items():
        # Make predictions on the test set
        y_pred = clf.predict(x_test)

        # Evaluate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        print(f'Test Accuracy of {clf} : \n-- {round(accuracy * 100, 2)}%')


bestparameter()
validation_score()

c:\iNeurOn_Projects\E2E_CreditFraudDetection\env\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best Parameters for log_reg: {'penalty': 'l1', 'C': 0.1}
Best Parameters for knears_neighbors: {'n_neighbors': 3, 'algorithm': 'brute'}
Best Parameters for svc: {'kernel': 'linear', 'C': 0.9}


c:\iNeurOn_Projects\E2E_CreditFraudDetection\env\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Parameters for tree_clf: {'min_samples_leaf': 5, 'max_depth': 3, 'criterion': 'entropy'}
Best Parameters for forest_clf: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}


Cross Validation Score of LogisticRegression(C=0.1, penalty='l1', solver='liblinear') : 
-- 94.54%
Cross Validation Score of KNeighborsClassifier(algorithm='brute', n_neighbors=3) : 
-- 93.39%
Cross Validation Score of SVC(C=0.9, kernel='linear') : 
-- 94.92%
Cross Validation Score of DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5) : 
-- 91.74%
Cross Validation Score of RandomForestClassifier(max_depth=30, n_estimators=200) : 
-- 94.41%

Test Accuracy of LogisticRegression(C=0.1, penalty='l1', solver='liblinear') : 
-- 91.88%
Test Accuracy of KNeighborsClassifier(algorithm='brute', n_neighbors=3) : 
-- 93.91%
Test Accuracy of SVC(C=0.9, kernel='linear') : 
-- 94.42%
Test Accuracy of DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samp